In [17]:
import re
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
from functools import partial
import matplotlib.pyplot as plt


In [18]:
keywords = [
    "made in america", "made in u.s.", "made in us",
    "american made", "usa made", "u.s. made", "us made",
    "buy american", "buy usa", "buy america",
    "support america", "support usa", "support u.s.",
    "patriot", "choose american", "choose usa", "choose u.s.", "choose america",
    "national pride", "usa based", "america based", "american based", "us based", "u.s. based",
    "usa produced", "america produced", "american produced", "us produced", "u.s. produced",
    "usa manufactured", "america manufactured", "american manufactured", "us manufactured", "u.s. manufactured",
    "american worker", "american job", "veteran owned", "veteran founded", "founded by veteran",
    "handcrafted in america", "handcrafted in usa", "handcrafted in u.s.", "handcrafted in us",
    "crafted in america", "crafted in u.s.", "crafted in us",
    "america heritage", "america tradition", "america value",
    "icon of america", "icon of usa", "icon of u.s.",
    "america manufactur", "u.s. manufactur"
]

In [22]:
# Load and process data
def load_and_process_file(file_path):
    def filter_columns(df):
        pattern = r"^\d{4}-\d{2}$"
        filtered_columns = ['std_name'] + [col for col in df.columns if re.match(pattern, col)]
        return df[filtered_columns]

    # Load only the first 100,000 rows for testing
    df = pd.read_csv(file_path, nrows=100000, low_memory=False)
    df = filter_columns(df)
    df = df.drop_duplicates(subset='std_name').sort_values(by='std_name').reset_index(drop=True)
    df = df.drop(columns=['std_name'])
    return df


In [23]:
df = load_and_process_file("buy_usa_second_round_with_additional_firms.csv")

In [24]:
df.head()

,2023-10,2023-09,2023-08,2023-07,2023-06,2023-05,2023-04,2023-03,2023-02,2023-01,...,1997-07,1997-06,1997-05,1997-04,1997-03,1997-02,1997-01,1996-12,1996-11,1996-10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,"14th Rose - Handbags, Purses, Woman's Bags\n F...",NaN,NaN,"14th Rose - Handbags, Purses, Woman's Bags\n F...",NaN,"14th Rose - Handbags, Purses, Woman's Bags\n F...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,About Soy. 1803 Candles - Best Scented Soy Can...,NaN,NaN,NaN,About Soy. 1803 Candles - Best Scented Soy Can...,About Soy. 1803 Candles - Best Scented Soy Can...,NaN,About Soy. 1803 Candles - Best Scented Soy Can...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Load and process data in chunks
def load_and_process_file(file_path):
    def filter_columns(df):
        pattern = r"^\d{4}-\d{2}$"
        filtered_columns = ['std_name'] + [col for col in df.columns if re.match(pattern, col)]
        return df[filtered_columns]

    df_chunk = pd.read_csv(file_path, chunksize=5000)
    results = []
    for chunk in df_chunk:
        chunk = filter_columns(chunk)
        chunk = chunk.drop_duplicates(subset='std_name').sort_values(by='std_name').reset_index(drop=True)
        chunk = chunk.drop(columns=['std_name'])
        results.append(chunk)
    
    return pd.concat(results, ignore_index=True)

# Keyword counting functions
def count_keywords_in_cell(cell, keywords):
    return sum(1 if keyword.lower() in str(cell).lower() else 0 for keyword in keywords)

def calculate_total_counts(df, columns, keywords):
    import pandas as pd  # Local import for multiprocessing context
    total_counts_df = pd.DataFrame(index=df.index, columns=columns, dtype=int).fillna(0)
    for index, row in df.iterrows():
        previous_count = 0
        for col_idx, col in enumerate(reversed(columns)):
            cell_value = row[col]
            current_count = count_keywords_in_cell(cell_value, keywords)
            if current_count == 0 and previous_count > 0:
                total_counts_df.at[index, columns[len(columns) - 1 - col_idx]] = previous_count
            else:
                total_counts_df.at[index, columns[len(columns) - 1 - col_idx]] = current_count
                previous_count = current_count
    return total_counts_df

# Parallel keyword existence calculation
def count_term_existence(df, columns, total_counts_df, keyword):
    import pandas as pd  # Local import for multiprocessing context
    term_existence = [0] * len(columns)
    for index, row in df.iterrows():
        previous_total_count = 0
        for col_idx, col in enumerate(reversed(columns)):
            cell_value = row[col]
            current_total_count = total_counts_df.at[index, col]
            if current_total_count == 0 and previous_total_count > 0:
                term_existence[len(columns) - 1 - col_idx] += 1
            elif pd.isna(cell_value) or isinstance(cell_value, int):
                term_existence[len(columns) - 1 - col_idx] += 0
            else:
                term_existence[len(columns) - 1 - col_idx] += 1 if keyword in str(cell_value).lower() else 0
            previous_total_count = max(current_total_count, previous_total_count)
    return {keyword: sum(term_existence)}

# Generate final value for each year with TF-IDF
def generate_final_value_by_year(df, total_counts_df, columns, document_count_sum, data):
    import pandas as pd  # Local import for multiprocessing context
    import math
    keyword, term_existence = data
    term_existence_full = [0] * len(columns)
    if term_existence == 0:
        return {keyword: term_existence_full}
    
    for index, row in df.iterrows():
        previous_value = 0
        for col_idx, col in enumerate(reversed(columns)):
            cell_value = row[col]
            current_total_count = total_counts_df.at[index, col]
            idf = math.log(1 + (document_count_sum / term_existence))
            keyword_count = str(cell_value).lower().count(keyword)
            value = (keyword_count * idf / len(str(cell_value))) * 100
            term_existence_full[len(columns) - 1 - col_idx] += value
            previous_value = value
    return {keyword: term_existence_full}

# Multiprocessing setup
def run_keyword_analysis(file_paths):
    all_tf_idf_totals = {}
    for file_path in file_paths:
        df = load_and_process_file(file_path)
        columns = list(df.columns)
        total_counts_df = calculate_total_counts(df, columns, keywords)
        document_count_sum = total_counts_df.sum().sum()
        
        func = partial(count_term_existence, df, columns, total_counts_df)
        with ProcessPoolExecutor(max_workers=4) as executor:
            term_counts = list(executor.map(func, keywords))

        term_count_dict = {k: v for d in term_counts for k, v in d.items()}
        func = partial(generate_final_value_by_year, df, total_counts_df, columns, document_count_sum)

        with ProcessPoolExecutor(max_workers=6) as executor:
            tf_idf_results = list(executor.map(func, ((keyword, term_count_dict[keyword]) for keyword in keywords)))

        tf_idf_total = {k: v for d in tf_idf_results for k, v in d.items()}
        all_tf_idf_totals[file_path] = tf_idf_total
    
    return all_tf_idf_totals

# Plotting function for all datasets on the same graph
def plot_results(all_tf_idf_totals, columns_trimmed):
    plt.figure(figsize=(10, 6))
    for label, tf_idf_total in all_tf_idf_totals.items():
        combined_values = [sum(values) for values in zip(*tf_idf_total.values())]
        plt.plot(columns_trimmed, combined_values, label=label)
    plt.legend()
    plt.title('TF-IDF Values Over Time for Different Data Sources')
    plt.xlabel('Year')
    plt.ylabel('TF-IDF Score')
    plt.show()

In [20]:
file_paths = [
    "buy_usa_second_round_with_additional_firms.csv",
    "about_us_second_round_with_additional_firms.csv",
    "company_website_second_round_with_additional_firms.csv"
]

In [21]:
all_tf_idf_totals = run_keyword_analysis(file_paths)
columns_trimmed = list(all_tf_idf_totals[file_paths[0]].values())[0]  # Use columns from the first dataset
plot_results(all_tf_idf_totals, columns_trimmed)

C:\Users\theal\AppData\Local\Temp\ipykernel_13108\2010235640.py:10: DtypeWarning: Columns (189,229,268,276,278,284,286,297,302,313,315,323,324,326,330,339) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in df_chunk:


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [ ]:
columns_trimmed = list(all_tf_idf_totals[file_paths[0]].values())[0]


In [ ]:
plot_results(all_tf_idf_totals, columns_trimmed)